# Segmenting and Clustering Neighborhoods in Singapore and Honolulu comparison (week4 & 5)

### Project goals: In this project, K-mean clustering will use on the study of Venues similarity.  First, a set of postal codes[2][6] obtain for the area of interests.  Next step will feed postal information into Foursqure.com for venues information.  The goal is to compare both cities what the historical influence, area of clustering on geolocation population and ethnicity similarity or if any.

###   This project compares Singapore and Honolulu at Hawaii.  Looking around the world, Singapore has it unique culture favor of \"East meets West\".  This country established less than a century ago in 1965.  Singapore has several distinct ethnic neighborhoods, including Katong, Kampong Glam, Geylang Serai, Chinatown and Little India[1].  As you can imagine such different ethic combination, it provides a breeding ground of culture melting pot. Before Singapore independent, British ruled Singapore between 1945 to 1964. A western culture introduced by that period of ruling.
   
###   City of the U.S. state of Hawaii, Honolulu has chosen in this project to compare with Singapore.  Selecting Honolulu as the comparing city but not other south east Asian city, it is because of the common similarities on both city .
   
###   Honolulu has similarity to Singapore on ethnic diversity, history, and ability of merging cultures.  Honolulu's geolocation is in the Pacific between the East and the West.  Naturally it becomes a merchant trading hub in 1800s [3].  Before Hawaii became one of 50th state in 1959, Honolulu was ruled by native Hawiian.  Kamehameha III had proclaimed Honolulu the capital city of his kingdom [5].  The major ethic group is Asian than white.  Other ethnicity groups are African American, American Indian or Alaska Native, Native Hawaiian and Other Pacific Islander[4]."
   
###   Both area has similar governing history of foreign power, ethnicity diversity, merchant hub, and many shoreline.  It is interesting to see food and stores where both places agglomerate to.

## Data perparation:
geonames.org logs postal code and geolocation on their site for everyone to download and reuse.  In the next cell, code was written to download and upzip the Singapore postal code data.  Page link here, http://download.geonames.org/export/zip/SG.zip, contains Singapore postal code information; http://download.geonames.org/export/zip/US.zip, contains United State postal code information:"

### Singapore data preparation

In [1]:
import wget
from zipfile import ZipFile

print('Beginning file download with wget module')

url = 'http://download.geonames.org/export/zip/SG.zip'
wget.download(url, 'geolocation/Singpostal.zip')

print("Waiting.... unzip file")
# Create a ZipFile Object and load sample.zip in it
with ZipFile('geolocation/Singpostal.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('geolocation/')
print('done')

Beginning file download with wget module
Waiting.... unzip file
done


In [17]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

path='geolocation/SG.txt'
df = pd.read_csv(path,sep='\t', header=None)

### To create the above dataframe:
The dataframe will purge the NaN columns in 0, 2-8 and 11 then assign the column name to 1:postalcode, 2:Neighborhood, 9:Latitude, and 10:Longitude

In [18]:
# Dataframe object is in df_list[0].  Assign df_list[0] to df the list first five rows
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,SG,18906,Straits Boulevard,NaN,NaN,NaN,NaN,NaN,NaN,1.2758,103.8496,NaN
1,SG,18907,Straits Boulevard,NaN,NaN,NaN,NaN,NaN,NaN,1.2749,103.8517,NaN
2,SG,18910,Marina Gardens Drive,NaN,NaN,NaN,NaN,NaN,NaN,1.2796,103.8690,NaN
3,SG,18915,Central Boulevard,NaN,NaN,NaN,NaN,NaN,NaN,1.2737,103.8601,NaN
4,SG,18916,Central Boulevard,NaN,NaN,NaN,NaN,NaN,NaN,1.2798,103.8515,NaN


In [19]:
df.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='int64')

In [20]:
# drop the NaN data columns
df.drop([0,3,4,5,6,7,8,11], axis=1, inplace=True)
#change cloumns name. Since it was not exist before, here just pass the cloumn 
df.columns = ['postalcode', 'Neighborhood', 'Latitude', 'Longitude']
df.head()

,postalcode,Neighborhood,Latitude,Longitude
0,18906,Straits Boulevard,1.2758,103.8496
1,18907,Straits Boulevard,1.2749,103.8517
2,18910,Marina Gardens Drive,1.2796,103.8690
3,18915,Central Boulevard,1.2737,103.8601
4,18916,Central Boulevard,1.2798,103.8515


### More than one neighborhood can exist in more than one postal code. For example, in the dataframe above, you will notice that neighborhood Straits Boulevard is listed twice and has two 'Latitude' and 'Longitude': Straits Boulevard appeared twice in geolocation. These two rows will be combined into one row with the 'Latitude', 'Longitude' mean value.  The postal code will be drop here which it doesn't provide necessary means for further analysis.

In [22]:
print(df.columns)
print(df.shape)

Index(['postalcode', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')
(121154, 4)


In [23]:
# use groupby on Neighborhood to reduce number of 'Latitude' and 'Longitude' value
df_g = df.groupby('Neighborhood')['Latitude','Longitude'].mean().reset_index()
#print(df_g.dtypes,'\n\n', df_g.head(),'\n\n',df_g.shape)
print('{0}\n\nDataframe size= {1}'.format(df_g.head(),df_g.shape))

    Neighborhood  Latitude   Longitude
0  Abingdon Road  1.368025  103.980287
1     Adam Drive  1.336363  103.814787
2      Adam Park  1.331032  103.812989
3      Adam Road  1.326481  103.813350
4      Adis Road  1.301325  103.847675

Dataframe size= (3863, 3)


### Foursquare developer account provides [500 Premium call](https://developer.foursquare.com/docs/api/troubleshooting/rate-limits) per day.  As this limit, to merge or cut down numbers of Neighborhood by Latitude, Longitude between two points within 1km is necessary.

### Sorting Latiude and longittude here to combin Neighorhoods by neighborhood distance

In [24]:
# helper function
# the diameter of the Earth is 12,742 km, unit result in this calculation is Km
# reference: http://mathforum.org/library/drmath/view/51816.html for latitude, longitude equation &
# https://stackoverflow.com/questions/42686300/how-to-check-if-coordinate-inside-certain-area-python

from math import cos, asin, sqrt
def distance(lat_set1, lon_set1, lat_set2, lon_set2):
    for lat1, lon1, lat2, lon2 in zip(lat_set1, lon_set1, lat_set2, lon_set2):
        p = 0.017453292519943295     #Pi/180
        a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a)) #distance_list #2*R*asin...; united in Km

# reference: https://stackoverflow.com/questions/34562261/get-pairwise-iterator-with-additional-item-in-the-end
from itertools import tee
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

### A better way to combin the Neighborhood is create a circle of interest area. Then test near point between the center point.  Code to check near neighborhood is in below and insprited from https://stackoverflow.com/questions/42686300/how-to-check-if-coordinate-inside-certain-area-python


In [25]:
numRun=centerControl=0;
borough=1.0;  #0.5 Km borough range
start = True;

print("Waiting... Calculating near Neighborhood and consolidate within {0} Km range".format(borough))
while start == True:
    rowdroplist=[]
    for (i1, row1), (i2, row2) in pairwise(df_g[numRun:].iterrows()): # given dataframe
        if centerControl==0: center=row1;  # set center point on row1 on each iterate
        else: row1=center;
        centerControl+=1;
        NeighborhoodDis = distance(lat_set1=[row1['Latitude']], lon_set1= [row1['Longitude']],\
                                   lat_set2=[row2['Latitude']], lon_set2= [row2['Longitude']])
        
        if NeighborhoodDis<= borough:
            rowdroplist.append(i2)
            combin_Neighborhood=', '.join((row1['Neighborhood'],row2['Neighborhood']))  # combin Neighborhood
            df_g.at[i1,'Neighborhood']=combin_Neighborhood  # change Neighborhood description

    numRun += 1;
    df_g.drop(rowdroplist, axis=0, inplace=True)
    centerControl=0; # reset centerControl to 0;
    if numRun > df_g.shape[0]: start = False;  # control while loop
    df_g.reset_index(drop=True, inplace=True)
    if numRun %50==1: print("Neighborhood consolidated {1} at counter {0}, Neighborhood remains:{2}" \
    .format(numRun,len(rowdroplist),df_g.shape[0]))
print("Number of Neighborhood after consolidating={0}'\n'".format(df_g.shape))

Waiting... Calculating near Neighborhood and consolidate within 1.0 Km range
Neighborhood consolidated 9 at counter 1, Neighborhood remains:3854
Neighborhood consolidated 11 at counter 51, Neighborhood remains:2296
Neighborhood consolidated 8 at counter 101, Neighborhood remains:1545
Neighborhood consolidated 11 at counter 151, Neighborhood remains:919
Neighborhood consolidated 8 at counter 201, Neighborhood remains:594
Neighborhood consolidated 0 at counter 251, Neighborhood remains:458
Neighborhood consolidated 10 at counter 301, Neighborhood remains:323
Number of Neighborhood after consolidating=(306, 3)'
'


In [26]:
df_g.head()

,Neighborhood,Latitude,Longitude
0,Abingdon Road,1.368025,103.980287
1,"Adam Drive, Adam Park",1.336363,103.814787
2,Adam Road,1.326481,103.813350
3,Adis Road,1.301325,103.847675
4,"Admiralty Drive, Admiralty Lane",1.449964,103.816195


## Honolulu data preparation

In [103]:
import wget
from zipfile import ZipFile

print('Beginning file download with wget module')

url = 'http://download.geonames.org/export/zip/US.zip'
wget.download(url, 'geolocation/USpostal.zip')

print("Waiting.... unzip file")
# Create a ZipFile Object and load sample.zip in it
with ZipFile('geolocation/USpostal.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('geolocation/')
print('done')

Beginning file download with wget module
Waiting.... unzip file
done


In [27]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

path='geolocation/US.txt'
df = pd.read_csv(path,sep='\t', header=None)

In [28]:
# extract honolulu postal code from US list
df_s=df[(df[3]=='Honolulu') | (df[2]=='Honolulu')|(df[5]=='Honolulu')]
df_s.shape

(69, 12)

### To create the above dataframe:
## The dataframe will purge the NaN columns in 7 and 8.   Columns 0,3,4,6 and 11 will drop then assign the column name to 1:postalcode, 2:Neighborhood, 5: area 9:Latitude, and 10:Longitude

In [29]:
# Dataframe object is in df_list[0].  Assign df_list[0] to df the list first five rows
df_s.head()

,0,1,2,3,4,5,6,7,8,9,10,11
8811,US,96701,Aiea,Hawaii,HI,Honolulu,3.0,NaN,NaN,21.3908,-157.9332,4.0
8812,US,96706,Ewa Beach,Hawaii,HI,Honolulu,3.0,NaN,NaN,21.3274,-158.0103,4.0
8813,US,96707,Kapolei,Hawaii,HI,Honolulu,3.0,NaN,NaN,21.3453,-158.0870,4.0
8814,US,96709,Kapolei,Hawaii,HI,Honolulu,3.0,NaN,NaN,21.3233,-158.0058,1.0
8815,US,96712,Haleiwa,Hawaii,HI,Honolulu,3.0,NaN,NaN,21.6312,-158.0693,1.0


In [30]:
df_s.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='int64')

In [31]:
# drop data columns
df_s.drop([0,3,4,6,7,8,11], axis=1, inplace=True)
#change cloumns name. Since it was not exist before, here just pass the cloumn 
df_s.columns = ['postalcode', 'Neighborhood', 'Area','Latitude', 'Longitude']
df_s.reset_index(drop=True, inplace=True)
df_s.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,postalcode,Neighborhood,Area,Latitude,Longitude
0,96701,Aiea,Honolulu,21.3908,-157.9332
1,96706,Ewa Beach,Honolulu,21.3274,-158.0103
2,96707,Kapolei,Honolulu,21.3453,-158.0870
3,96709,Kapolei,Honolulu,21.3233,-158.0058
4,96712,Haleiwa,Honolulu,21.6312,-158.0693


### Honolulu postal code is sparse.  Consolidate postal code in area is not necessary.

## Discussion on the data

### Singapore

### Once obtain both cities postal code tables, one can found that Singapore's postal code divided into fine grind.  It's table contains 121,154 rows.  Examining neighborhood, one street name can divide in multiple postal codes that each one assign a geolocation position.  If put a same street name with two geolactions, one can calculate a distance between two geolocations.  Some distance on those points are within 200~500m (meter).  A grouping is necessary to reduce the geolocation points.,
### Pandas dataframe groupby method was used on 'Neighborhood' which resulted rows consolidating to 3863. records.  It is quick large for Foursquare developer API call for this project.  it requires further geolocation points reduction.  A simply and direct way is to average near two postal location and to join the neighborhood.  The coding is straight forward.  However, this approach created two disadvantage, geolocation will shift, first.  Second, the geolocation point reduction is not optimum on result.  Some of the geolocations are near each other but the algorithm will not detected since geolocation shift will shift after previous result is records.  Two or three recalculating might occur during geolocation reduction computing.,
### A better way is calculation a set radius circle which an arbitrate geolocation point selects as a center then computing a list a distance respecting to center.  If a distance within the circle, then consolidate those neighborhoods.  The center will not shift on this algorithm.  After the first pass, next center will choose on the second row on reduced dataset.  It will run maximum n/2 time if the dataframe contain n rows (which means none of geolocation fall in each other in the circle area).  Therefore, the algorithm will expect run time less (n/2)^2

## Honolulu

### Honolulu postal code data set is straight forward since the geolocation point is sparsed according postal code assignment.  It only has 69 rows. No consolidation was done.  At one time, an idea to attempt on geolocation expansion was considered.  This can be done as reversing the step in Singapore dataset operation using a distance expansion to more geolocation points.  However, some of new geolocation points will fall outside to sea or rural area.  It does not serve the purpose to manual increase extra geolocation points. 

# Visualization geolocations

## Singapore

In [33]:
import folium # map rendering library

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Lookup Singapore geolocation

In [34]:
latitude =df_g['Latitude'][0]
longitude = df_g['Longitude'][0]
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.368025, 103.98028749999999.


Create Singapore area postal MAP. Circle spots are the postal code geo-location. This is not the clustering map!

In [35]:
# create map of Manhattan using latitude and longitude values
map_SingaporeRegion = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_g['Latitude'], df_g['Longitude'], df_g['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SingaporeRegion)  
    
map_SingaporeRegion

## Honolulu

In [37]:
latitude =df_s['Latitude'][0]
longitude = df_s['Longitude'][0]
print('The geograpical coordinate of honolulu are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of honolulu are 21.3908, -157.9332.


Create honolulu area postal MAP. Circle spots are the postal code geo-location. This is not the clustering map!

In [38]:
# create map of Manhattan using latitude and longitude values
map_honoluluRegion = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_s['Latitude'], df_s['Longitude'], df_s['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_honoluluRegion)  
    
map_honoluluRegion

### Reference:
1. https://en.wikipedia.org/wiki/Culture_of_Singapore.\
2. https://en.wikipedia.org/wiki/Postal_codes_in_Singapore\
3. https://en.wikipedia.org/wiki/Honolulu\
4. Profile of General Population and Housing Characteristics: 2010 Demographic Profile Data (DP-1): Urban Honolulu CDP, Hawaii. U.S. Census Bureau, American Factfinder. Retrieved May 25, 2012.
5. "Honolulu History, Hawaii". Travelgrove.com. Retrieved May 22, 2012.
https://www.geonames.org/postal-codes/US/HI/003/honolulu.html
6. geolocation http://download.geonames.org/export/zip/
    